In [26]:
from ase import Atoms
from ase.build import molecule
import numpy as np
from scipy.spatial.transform import Rotation
from ase.visualize import view

class RandomMoleculeStructure:
    def __init__(self, molecule_templates, num_molecules_per_type, cell_size, overlap_threshold=2.0):
        self.molecule_templates = molecule_templates
        self.num_molecules_per_type = num_molecules_per_type
        self.cell_size = cell_size
        self.overlap_threshold = overlap_threshold
        self.positions = np.empty((0, 3))
        

    def random_rotation_matrix(self):
        """
        ランダムな回転行列を生成する。
        """
        random_rotation = Rotation.from_rotvec(np.random.rand(3) * 2 * np.pi)
        return random_rotation.as_matrix()

    def check_overlap(self, new_pos):
        """
        分子同士の重なりをチェックし、新しい位置が重なる場合はTrueを返す。
        """
        dists = np.linalg.norm(self.positions - new_pos, axis=1)
        return np.any(dists < self.overlap_threshold)

    def generate_structure(self):
        """
        分子をランダムな向きに回転させて配置する。
        """
        for mol_index, mol_template in enumerate(self.molecule_templates):
            while len(self.positions) < sum(self.num_molecules_per_type[:mol_index + 1]):
                new_pos = np.random.rand(3) * self.cell_size
                if not self.check_overlap(new_pos):
                    self.positions = np.vstack([self.positions, new_pos])

        large_structure = Atoms(cell=self.cell_size)
        for i, pos in enumerate(self.positions):
            mol_index = 0
            while i >= sum(self.num_molecules_per_type[:mol_index + 1]):
                mol_index += 1
            mol_template = self.molecule_templates[mol_index]
            mol = mol_template.copy()
            self.rotation_matrix = self.random_rotation_matrix()
            mol.positions = np.dot(mol.positions, self.rotation_matrix.T)
            mol.translate(pos)
            large_structure += mol

        large_structure.set_pbc([True, True, True])
        return large_structure

# 使用例
num_molecules_per_type = [500, 130, 200]
cell_size = [100, 100, 100]
water_molecule = molecule('H2O')
carbon_dioxide_molecule = molecule('CO2')
methane_molecule = molecule('CH4')

molecule_templates = [water_molecule, carbon_dioxide_molecule, methane_molecule]

random_structure = RandomMoleculeStructure(molecule_templates, num_molecules_per_type, cell_size)
structure = random_structure.generate_structure()
print(structure)
# view(structure)

Atoms(symbols='C330H1800O760', pbc=True, cell=[100.0, 100.0, 100.0])


In [27]:
from ase.build import (
    fcc100, fcc110, fcc111, fcc211, fcc111_root,
    bcc100, bcc110, bcc111, bcc111_root,
    hcp0001, hcp10m10, hcp0001_root,
    diamond100, diamond111
)

pt100_atoms = fcc100("Pt", size=(10, 10, 5), vacuum=0.0)
# view(pt100_atoms)
print(pt100_atoms.cell[0])

[27.71858582  0.          0.        ]


In [28]:
num_molecules_per_type = [200, 200, 200]
cell_size = [pt100_atoms.cell[0][0], pt100_atoms.cell[1][1], 30]
water_molecule = molecule('H2O')
carbon_dioxide_molecule = molecule('CO2')
methane_molecule = molecule('CH4')

molecule_templates = [water_molecule, carbon_dioxide_molecule, methane_molecule]

random_structure = RandomMoleculeStructure(molecule_templates, num_molecules_per_type, cell_size)
structure = random_structure.generate_structure()
print(structure)
# view(structure)

Atoms(symbols='C400H1200O600', pbc=True, cell=[27.71858582251266, 27.71858582251266, 30.0])


In [34]:
slab_cell = pt100_atoms.get_cell()
# structure.translate(slab_cell[0] / 2 + slab_cell[1] / 2+ slab_cell[2] / 2)
slab_center = np.mean(pt100_atoms.positions, axis=0)+structure.cell[2]/2+slab_cell[2] / 2+3
structure.translate(slab_center - np.mean(structure.positions, axis=0))
merged=pt100_atoms+structure
merged.set_cell((slab_cell[0], slab_cell[1], slab_cell[2]+structure.cell[2]+[0,0,40]))
merged.wrap()
view(merged)
print(len(merged))

2700


In [30]:
slab_center

array([16.02773534, 16.02773534, 25.84      ])